In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/barbara/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def get_unique_words(text):
    words = [word for message in text for word in message]
    return set(words)

def co_occurence_matrix_with_window(text, window_size, stop_words):
    '''
    Calculates the co-occurence matrix using a sliding window of size window_size as context    
    Output is (unique) words x (unique) words
    '''
    unique_words = get_unique_words(text)
    n = len(unique_words) #number of unique words
    co_matrix = pd.DataFrame(data=np.zeros([n,n]), columns=unique_words, index=unique_words, dtype='int')

    #for each word in each message, counts +1 for each pair [word,context]
    for message in text:
        msg_len = len(message)
        for i, word in enumerate(message):
            first = max(i - window_size, 0)
            last = min(i + window_size + 1, msg_len)
            for context in message[first:last]:
                co_matrix.loc[word,context] += 1
    return co_matrix  
    
def word_occurence_matrix(text, target=None, stop_words=None, binary=True, preprocess_text=False):  
    '''
    Output is messages x (unique) words
    
    If binary=True, then each element represents if the word is in the message or not.
    Otherwise, it represents the count of how many times that word appears in that message.
    ''' 
    if target:
        text = list(filter(lambda x : target in x, text)) #Filter comments in which target word is present
        
    preprocessor = CountVectorizer(strip_accents='unicode').build_preprocessor()   
    if stop_words:        
        stop_words = [preprocessor(word) for word in stop_words] #preprocesses stop words
    if preprocess_text:
        text = [preprocessor(msg) for msg in text] #preprocesses text
        
    #calculates word count for each message
    vectorizer = CountVectorizer(strip_accents='unicode', stop_words=stop_words, binary=binary)
    X = vectorizer.fit_transform(text).toarray()
    
    labels = vectorizer.get_feature_names()
    
    return X, labels

def co_occurence_matrix(word_occurence_matrix):
    '''
    Co-occurence matrix created based on the word count/occurence matrix.     
    In other words, this co-occurence matrix will consider each message as the context for all words in it.
    '''
    co_X = word_occurence_matrix.T @ word_occurence_matrix
    np.fill_diagonal(co_X, 0)
    return co_X

def score_ngrams(text, freq_filter=1, score_metric=BigramAssocMeasures().pmi):
    '''
    Score n-grams using score_metric. Defaults to PMI.
    '''
    words = [word for msg in text for word in msg.split()]
    finder = BigramCollocationFinder.from_words(words)
    finder.apply_freq_filter(freq_filter)
    return finder.score_ngrams(score_metric)

### Dados

In [4]:
comments = pd.read_csv('../../dados/instagram/instagram_comments.csv')
stop_words = [word.rstrip() for word in open('../stopwords.txt')]

#Preprocesses text
preprocessor = CountVectorizer(strip_accents='unicode').build_preprocessor() #lowercase and strip accents
stop_words = [preprocessor(word) for word in stop_words]
comments['texto_do_comentario'] = [preprocessor(msg) for msg in comments['texto_do_comentario']]
comments['texto_do_comentario'] = [' '.join([word for word in RegexpTokenizer(r'\w+').tokenize(msg) 
                                             if not word in stop_words])
                                   for msg in comments['texto_do_comentario']]

text = comments['texto_do_comentario'].dropna()

N = 30 #Consider only the N most frequent words

In [5]:
comments_by_channel = []
channels = []
for channel, group in comments.groupby('dono_do_post'):
    channel_comments = group['texto_do_comentario']
    comments_by_channel.append(channel_comments)
    channels.append(channel)

### 1. Co-ocorrência 

#### 1.1. Co-cocorrência com uma palavra pré-definida

In [6]:
target = 'cloroquina'

X, labels = word_occurence_matrix(text, target=target, stop_words=stop_words)

word_count = X.sum(axis=0) #no. of comments containing A in which each word appears
word_count = sorted(zip(labels, word_count), reverse=True, key=lambda tupl : tupl[1])
print(f'{N} palavras que mais ocorrem nos mesmos comentários em que "{target}" ocorre:\n')
print(word_count[:N])

30 palavras que mais ocorrem nos mesmos comentários em que "cloroquina" ocorre:

[('cloroquina', 99), ('hidroxicloroquina', 38), ('dr', 26), ('sobre', 25), ('covid', 22), ('uso', 21), ('saude', 17), ('tratamento', 17), ('19', 15), ('azitromicina', 15), ('lair', 15), ('medico', 14), ('pra', 14), ('ivermectina', 13), ('porque', 13), ('vai', 12), ('bom', 11), ('medicos', 11), ('mundo', 11), ('remedio', 11), ('falar', 10), ('opiniao', 10), ('presidente', 10), ('todo', 10), ('casa', 9), ('contra', 9), ('doenca', 9), ('doutor', 9), ('efeitos', 9), ('gente', 9)]


In [7]:
cc_by_channel = []
cc_channels = []
cc_empty = []
for i, channel in enumerate(comments_by_channel):
    try:
        X, labels = word_occurence_matrix(channel, target=target, stop_words=stop_words)
        word_count = X.sum(axis=0) #no. of comments containing A in which each word appears
        word_count = sorted(zip(labels, word_count), reverse=True, key=lambda tupl : tupl[1])
        cc_by_channel.append(word_count)
        cc_channels.append(channels[i])
    except: #target word doesn't occur in the channel
        cc_empty.append(channels[i])
    

df_cc = pd.DataFrame(cc_by_channel, index = cc_channels).T
print(f'Nenhuma ocorrência de "{target}" nos perfis: ', cc_empty)
print(f'Palavras que mais co-ocorrem nos comentários com "{target}" em cada perfil:')
df_cc.head(10)

Nenhuma ocorrência de "cloroquina" nos perfis:  ['dr.galvao', 'juliommais']
Palavras que mais co-ocorrem nos comentários com "cloroquina" em cada perfil:


,dralaindutra,drfelipeades,drfernandoneuro,drlairribeiro,imedlkep,sitedrauziovarella
0,"(cloroquina, 6)","(cloroquina, 6)","(cloroquina, 13)","(cloroquina, 20)","(hidroxicloroquina, 5)","(cloroquina, 52)"
1,"(hidroxicloroquina, 6)","(pacientes, 3)","(dr, 6)","(lair, 15)","(boa, 3)","(hidroxicloroquina, 13)"
2,"(ivermectina, 5)","(cancer, 2)","(hidroxicloroquina, 6)","(dr, 12)","(ivermectina, 3)","(sobre, 10)"
3,"(beneficios, 3)","(medicos, 2)","(uso, 5)","(hidroxicloroquina, 8)","(assim, 2)","(azitromicina, 9)"
4,"(medicos, 3)","(sobre, 2)","(19, 4)","(sobre, 8)","(cloroquina, 2)","(covid, 9)"
5,"(tratamento, 3)","(uns, 2)","(covid, 4)","(ribeiro, 7)","(covid, 2)","(porque, 9)"
6,"(dralaindutra, 2)","(19, 1)","(fernando, 4)","(uso, 7)","(dra, 2)","(19, 8)"
7,"(estudos, 2)","(3dias, 1)","(sobre, 4)","(todo, 6)","(hidroxi, 2)","(pra, 8)"
8,"(medico, 2)","(acometido, 1)","(ar, 3)","(azitromicina, 5)","(medicos, 2)","(casa, 7)"
9,"(muitos, 2)","(acredito, 1)","(bem, 3)","(bom, 5)","(noite, 2)","(cura, 7)"


#### 1.2. N-gramas

Considerações:

- PMI - favorece n-gramas raros. Muitos n-gramas com a mesma score (a mais alta)

#### PMI

In [8]:
pmi = BigramAssocMeasures().pmi
print(f'{N} com score mais alta:')
score_ngrams(text, score_metric=pmi, freq_filter=10)[:N]

30 com score mais alta:


[(('jnr', '_vasconcelos'), 17.374932488156595),
 (('97130', '1312'), 17.237428964406654),
 (('ncbi', 'nlm'), 17.1118980823228),
 (('nlm', 'nih'), 17.1118980823228),
 (('claudiana', 'mesquitta'), 17.111898082322796),
 (('sirio', 'libanes'), 17.111898082322796),
 (('tulio', 'gadelha'), 17.099925440656722),
 (('assine', 'newsletter'), 16.99642086490286),
 (('natifonseca', 'vidafit'), 16.99642086490286),
 (('birgit', 'jano'), 16.88950566098635),
 (('emily', 'lobo80'), 16.88950566098635),
 (('luan', 'sousa15'), 16.88950566098635),
 (('rebecca', 'hansen'), 16.88950566098635),
 (('margareth', 'souzaa'), 16.870889982819),
 (('fisio_cris', 'arantes'), 16.789969987435438),
 (('mayra_roman', 'seg'), 16.789969987435438),
 (('arco', 'iris'), 16.774028443566415),
 (('hansen', '9406'), 16.774028443566415),
 (('viniciussss', 'bernardo'), 16.696860583043957),
 (('sund', 'ay9426'), 16.696860583043954),
 (('tina', 'ferreiraa'), 16.62647125515256),
 (('lap', '1983'), 16.617909241649134),
 (('DE', 'FITOTER

In [9]:
pmi_by_channel = []
for i, channel in enumerate(comments_by_channel):
    pmi_scores = score_ngrams(channel, score_metric=pmi, freq_filter=10)
    pmi_by_channel.append(pmi_scores)

df_cc = pd.DataFrame(pmi_by_channel, index = channels).T
df_cc.head(10)

,dr.galvao,dralaindutra,drfelipeades,drfernandoneuro,drlairribeiro,imedlkep,juliommais,sitedrauziovarella
0,"((dr, galvao), 4.215349144332329)","((97130, 1312), 12.622925360775032)","((alexsandre, moreira), 11.986294567501918)","((arco, iris), 15.926193667425753)","((luan, sousa15), 13.940764056146941)","((boa, noite), 7.868854044009884)","((continue, abencoando), 6.0175187209047944)","((emily, lobo80), 16.00296440977047)"
1,"((parabens, dr), 3.0760124627234404)","((assine, newsletter), 12.622925360775032)","((claudiana, mesquitta), 11.986294567501918)","((hansen, 9406), 15.810716450005817)","((tina, ferreiraa), 13.940764056146941)","((deus, abencoe), 7.529367577738219)","((deus, abencoe), 4.157431153280667)","((jnr, _vasconcelos), 16.00296440977047)"
2,None,"((einstein, visite), 12.622925360775032)","((atividade, fisica), 11.638371264081613)","((mayra_roman, seg), 15.810716450005817)","((vanda, scalabrini), 13.815233174063081)","((lucy, kerr), 7.277113010657665)","((deus, continue), 4.099980881096769)","((lap, 1983), 16.00296440977047)"
3,None,"((gratuitos, semanais), 12.622925360775032)","((alcool, gel), 11.275801184696906)","((rebecca, hansen), 15.810716450005817)","((natifonseca, vidafit), 13.699755956643145)","((gostaria, saber), 7.091246465346334)","((parabens, deus), 1.3542980698294524)","((fisio_cris, arantes), 15.739930003936678)"
4,None,"((margareth, souzaa), 12.622925360775032)","((levando, serio), 10.085244850482464)","((birgit, jano), 15.703801246089306)","((integrando, terapias), 13.689702291979224)","((dose, unica), 6.618139909811958)",None,"((minas, gerais), 15.739930003936678)"
5,None,"((conexao, cerebrointestino), 12.497394478691174)","((efeitos, colaterais), 10.029280492467567)","((buzdugan, 940), 15.511156168146915)","((low, carb), 13.584278739223208)","((dra, lucy), 5.951068807298075)",None,"((tulio, gadelha), 15.727957362270601)"
6,None,"((1312, cuidados), 12.275002057354726)","((covid, 19), 9.895235902675198)","((sund, ay9426), 15.511156168146915)","((sarah, chaves), 13.477363535306697)","((15, dias), 5.943212274104228)",None,"((mallmann, rosane), 15.418001909049316)"
7,None,"((cuidados, rayane), 12.275002057354726)","((pacientes, oncologicos), 9.157244574344999)","((dradebora, tonetti), 15.50472989898748)","((terapias, sarah), 13.467309870642776)","((3, dias), 5.647756390578056)",None,"((dradebora, tonetti), 15.324892504657832)"
8,None,"((alainuro, assine), 12.175466383803812)","((triplo, negativo), 9.109992291726824)","((nelia, lopez), 15.42369332689657)","((viniciussss, bernardo), 13.400195674784241)","((uso, ivermectina), 5.451142410432716)",None,"((youtu, be), 15.237429663407491)"
9,None,"((patologia, 20anos), 12.175466383803812)","((isolamento, social), 8.934764266861839)","((elena, buzdugan), 15.341231166704603)","((royal, prestige), 13.40019567478424)","((pode, tomar), 4.040620392276367)",None,"((guilherme, padua), 15.199361622573976)"


#### likelihood ratio

In [10]:
lr = BigramAssocMeasures().likelihood_ratio
print(f'{N} com score mais alta:')
score_ngrams(text, score_metric=lr, freq_filter=10)[:N]

30 com score mais alta:


[(('dr', 'fernando'), 69781.29586736573),
 (('deus', 'abencoe'), 31756.003282794674),
 (('bom', 'dia'), 29388.273716842567),
 (('dr', 'drauzio'), 25847.964936781264),
 (('boa', 'noite'), 23780.222137181445),
 (('boa', 'tarde'), 15032.334278176995),
 (('dr', 'lair'), 14656.218827188624),
 (('god', 'bless'), 13147.650342311243),
 (('lair', 'ribeiro'), 12761.662031855476),
 (('gomes', 'pinto'), 12638.826145094372),
 (('thank', 'you'), 12614.205243654347),
 (('you', 'are'), 9748.713132357845),
 (('fernando', 'gomes'), 9425.17584053534),
 (('covid', '19'), 8558.578274936293),
 (('pedir', 'desculpas'), 8491.565033018751),
 (('feliz', 'natal'), 7435.309219596484),
 (('good', 'morning'), 7320.843325271788),
 (('don', 't'), 7099.191953023001),
 (('it', 's'), 6371.580770651734),
 (('drauzio', 'varella'), 6277.39700738431),
 (('happy', 'birthday'), 6216.99456787011),
 (('todo', 'respeito'), 5882.266572997278),
 (('bless', 'you'), 5637.831461654684),
 (('parabens', 'dr'), 5610.11550807307),
 (('me

In [11]:
lr_by_channel = []
for i, channel in enumerate(comments_by_channel):
    lr_scores = score_ngrams(channel, score_metric=lr, freq_filter=10)
    lr_by_channel.append(lr_scores)

df_cc = pd.DataFrame(lr_by_channel, index = channels).T
df_cc.head(10)

,dr.galvao,dralaindutra,drfelipeades,drfernandoneuro,drlairribeiro,imedlkep,juliommais,sitedrauziovarella
0,"((dr, galvao), 77.46390660201422)","((vitamina, d), 2653.4033271977587)","((dr, felipe), 2359.9896315459337)","((dr, fernando), 70992.03694595296)","((dr, lair), 15786.460174543648)","((dra, lucy), 225.3395821942578)","((deus, abencoe), 213.71566859289396)","((dr, drauzio), 30030.066210710727)"
1,"((parabens, dr), 39.565776269844086)","((dr, alain), 1740.5089753572047)","((deus, abencoe), 956.0827919417234)","((bom, dia), 27522.75830834331)","((lair, ribeiro), 8250.893904143306)","((dose, unica), 182.58572708817792)","((continue, abencoando), 150.02351441547142)","((pedir, desculpas), 7323.1263412113085)"
2,None,"((deus, abencoe), 857.9690974800146)","((triplo, negativo), 884.1124086793337)","((deus, abencoe), 20983.1771657826)","((deus, abencoe), 2556.0329781005776)","((boa, noite), 131.84871744042397)","((deus, continue), 129.73010172467957)","((deus, abencoe), 5727.9912166057975)"
3,None,"((canal, youtube), 592.5210695287567)","((boa, viagem), 684.1014247481047)","((boa, noite), 18813.080393199343)","((oleo, coco), 2508.3949456950754)","((lucy, kerr), 129.26176780291)","((parabens, deus), 8.371938194330339)","((drauzio, varella), 5047.018974806956)"
4,None,"((covid, 19), 582.8125964036387)","((cancer, mama), 653.1266297286314)","((boa, tarde), 11938.202172884683)","((todos, dias), 1731.7579897580695)","((deus, abencoe), 119.26577920648833)",None,"((covid, 19), 3925.7199640599865)"
5,None,"((todos, dias), 486.2423986637534)","((parabens, dr), 549.414192212053)","((god, bless), 11626.097282802613)","((feliz, natal), 1639.4579906239824)","((15, dias), 94.47044886683082)",None,"((humano, incrivel), 3443.9451623247714)"
6,None,"((oleo, coco), 461.6469755465177)","((grupo, risco), 498.82735790303735)","((gomes, pinto), 11289.78456755531)","((bodyfood, nutricao), 1028.1544732958446)","((gostaria, saber), 92.0754160929672)",None,"((todo, respeito), 3074.860754269629)"
7,None,"((gostaria, saber), 445.76709326363925)","((boa, noite), 407.4097118486861)","((thank, you), 10447.628475730704)","((ministro, saude), 1003.2484897180091)","((3, dias), 86.91861336871553)",None,"((rede, globo), 2965.5427516064847)"
8,None,"((boa, noite), 423.5092365501387)","((gracas, deus), 399.52731641238654)","((fernando, gomes), 7830.214368729864)","((nutricao, funcional), 994.9385001966921)","((uso, ivermectina), 71.63931827592812)",None,"((seres, humanos), 2918.972117139524)"
9,None,"((alain, dutra), 419.36598521903625)","((feliz, natal), 398.679886765132)","((you, are), 7594.457971744974)","((covid, 19), 952.9540306150559)","((tomar, ivermectina), 58.62809418603622)",None,"((grupo, risco), 2822.1360856064966)"
